In [15]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [16]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
init_lending_data_df = pd.read_csv(
    "Resources/lending_data.csv"
)

# Review the DataFrame
init_lending_data_df.head(5)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [17]:
# Separate the data into labels and features (Part 1)

# Separate the y variable, the labels
y = pd.DataFrame({"loan_status":init_lending_data_df["loan_status"]})

# Review the y variable Series
y.head(5)


,loan_status
0,0
1,0
2,0
3,0
4,0


In [18]:
# Separate the data into labels and features (Part 2)

# Separate the X variable, the features
X = init_lending_data_df.drop(["loan_status"],axis=1)

# Review the X variable DataFrame
X.head(5)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [19]:
# Check the balance of our target values
y["loan_status"].value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [20]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
 y,
 # Assign a random_state of 1 to the function
 random_state=1
 )


---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [21]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
classifier = LogisticRegression(
    # Assign a random_state parameter of 1 to the model
    random_state=1
    )

In [25]:
# Fit the model using training data
classifier.fit(X_train, np.array(y_train).reshape(-1,))

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [26]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [28]:
# Print the balanced_accuracy score of the model
bal_acc_score = balanced_accuracy_score(y_test,predictions)
print(f'The Balanced Accuracy Score is {bal_acc_score}.')

The Balanced Accuracy Score is 0.9520479254722232.


In [29]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, predictions)

array([[18663,   102],
       [   56,   563]], dtype=int64)

In [31]:
# Print the classification report for the model
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model predicts the '0' (healthy loan) label with 100% accuracy given the testing portion of the data, but the `1` (high-risk loan) label with only an 85% accuracy. This is likely due to the high rate of false positives (102 of the 665).

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [35]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
ros_model = RandomOverSampler(
    # Assign a random_state parameter of 1 to the model
    random_state=1
)

In [44]:
# Fit the original training data to the random_oversampler model and resample the data
X_res, y_res = ros_model.fit_resample(X_train, y_train)

In [59]:
# Count the distinct values of the resampled labels data
y_res["loan_status"].value_counts()

1    56271
0    56271
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [61]:
# Instantiate the Logistic Regression model
res_classifier = LogisticRegression(
    # Assign a random_state parameter of 1 to the model
    random_state=1
    )

In [63]:
# Fit the model using the resampled training data
res_classifier.fit(X_res,np.array(y_res).reshape(-1,))

LogisticRegression(random_state=1)

In [64]:
# Make a prediction using the testing data
res_predictions = res_classifier.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [65]:
# Print the balanced_accuracy score of the model 
res_bal_acc_score = balanced_accuracy_score(y_test,res_predictions)
print(f'The Balanced Accuracy Score is {res_bal_acc_score}.')

The Balanced Accuracy Score is 0.9936781215845847.


In [66]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, res_predictions)

array([[18649,   116],
       [    4,   615]], dtype=int64)

In [67]:
# Print the classification report for the model
print(classification_report(y_test, res_predictions))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.99      0.91       619

    accuracy                           0.99     19384
   macro avg       0.92      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model with oversampled data predicts the '0' (healthy loan) label with 100% accuracy given the testing portion of the data, but the `1` (high-risk loan) label with only an 84% accuracy. This is likely due to the high rate of false positives (116 of the 731), even with the extra data.